# Fresh Grad Job Ads in Penang



Author: Subi

Date: 4/26


Purpose: To Find the Latest Situation of Job Market for Fresh Grad in Penang


Methodology: Selenium, BeautifulSoup, Pandas,Numpy, Matplotlib, Plotly


Findings: Most job category are from Sales-Financial Services while most industry are from Banking/Finance, Most company are willing to offer 2.5k as a minimum starting salary and 5k as max starting salary


Recommendation/Conclusion: can get lots of data if we include other state or expand the sample, can check relationship between each variable such as the relationship between industry and job category. Can include more variable if can get company size to cross check with the salary they willing to offer. 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time 
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
from webdriver_manager.chrome import ChromeDriverManager



page_url = []
max_page = 7
for i in range(1,max_page + 1):
    page_num = str(i)
    url = 'https://www.jobstreet.com.my/en/job-search/campus/job-vacancy.php?area=1&option=1&location=50700&job-type=1%7C2%7C4%7C8%7C32&job-source=1%2C64&classified=1&job-posted=0&src=45&pg=' + page_num + '&sort=1&order=0&srcr=12&vtype=list'
    page_url.append(url)

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://myjobstreet.jobstreet.com.my/home/login.php?site=my&language_code=3&go=JOB-ADS&nrfr=1')
   
user = browser.find_element_by_id('login_id')
user.send_keys('#########@yahoo.com') #enter ur email

#same for pw
pw = browser.find_element_by_id('password')
pw.send_keys('########') #enter ur pasword

#send user name &keys
pw.send_keys(Keys.RETURN)


Trying to download new driver from http://chromedriver.storage.googleapis.com/81.0.4044.69/chromedriver_win32.zip
Unpack archive C:\Users\mohammadsubhi.jamil\.wdm\drivers\chromedriver\81.0.4044.69\win32\chromedriver.zip


In [5]:
#define get fn

def getAndParseURL(url):
    browser.get(url)
    code = browser.page_source
    soup = BeautifulSoup(code,'html.parser')
    return(soup)

In [6]:
#Define scrape function for each intended column

def job(url):
    soup = getAndParseURL(url)
    soup_list = soup.findAll(class_ = 'rRow')
    return(soup_list.find(class_='rRowTitle').get_text() for soup_list in soup_list)


def company(url):
    soup = getAndParseURL(url)
    soup_list = soup.findAll(class_ = 'rRow')
    return(soup_list.find('h3').get_text() for soup_list in soup_list)

def category(url):
    soup = getAndParseURL(url)
    soup_list = soup.findAll(class_ = 'rRow')
    return(soup_list.find(class_='smalldesc').get_text() for soup_list in soup_list)


def industry(url):
    soup = getAndParseURL(url)
    soup_list = soup.findAll(class_ = 'rRow')
    Company1 = [soup_list.find(class_='colComp') for soup_list in soup_list]
    return(Company1[items].find("div", attrs={"class":"smalldesc"}).text.replace('Industry:','') for items in range(len(Company1)))


def salary(url):
    soup = getAndParseURL(url)
    soup_list = soup.findAll(class_ = 'rRow')
    return(soup_list.find(class_='colSal').get_text().replace('\n','').replace('MYR','').replace(',','') for soup_list in soup_list)

In [7]:
#scrape each column and put it into list

Job_title = []
Company = []
Category = []
Industry = []
Salary = []
for url in page_url:
    time.sleep(10)
    Job_title.extend(job(url))
    Company.extend(company(url))
    Category.extend(category(url))
    Industry.extend(industry(url))
    Salary.extend(salary(url))
    
browser.close() 

In [8]:
#Put everything into Pandas df

Jobstreet = pd.DataFrame({
    'Job_Title':Job_title,
    'Company':Company,
    'Job_Category':Category,
    'Industry': Industry,
    'Salary':Salary,  
    
})

In [9]:
df = Jobstreet

In [10]:
#clean and prep the data
#split salary into two section

df['Min_Salary'], df['Max_Salary'] = df['Salary'].str.split('-', 1).str #since we have range, split column salary by '-' to min max
df = df.drop(['Salary'], 1) #drop Salary

In [11]:
df = df.replace(r'^\s*$', np.nan, regex=True) # replace blank space with nan
df = df.replace(r'Below Expected Salary', np.nan, regex=True) 
df = df.replace(r'Around Expected Salary', np.nan, regex=True)
df = df.replace(r'Above Expected Salary', np.nan, regex=True)# replace 'Below Expected Salary ' with nan

In [12]:
df

,Job_Title,Company,Job_Category,Industry,Min_Salary,Max_Salary
0,IT Sales Executive,SiteGiant Sdn. Bhd.,Sales - Eng/Tech/IT,IT / Software,2500,4000
1,Electronic Technician,Agensi Pekerjaan NCS Solutions Sdn. Bhd.,Electronics Engineering > Electronics Technici...,HR Mgmt / Consulting,1800,2400
2,Product Specialist,Sun Pharmaceutical Sdn Bhd,Sales - Corporate,BioTech / Pharmaceutical,2200,3200
3,Business Development Coordinator,UBCT INDUSTRIAL SOLUTION SDN.BHD.,Marketing/Business Dev,Manufacturing / Production,1800,2500
4,Software Engineering Graduate Trainee,Intel Technology Sdn. Bhd.,IT - Software > Software Engineer/Programmer,Semiconductor,NaN,NaN
5,SALES EXECUTIVE / REPRESENTATIVE (Penang / Kua...,Panasonic Malaysia Sdn Bhd,Retail Sales,Electrical & Electronics,NaN,NaN
6,MYS CTSM PG - AML FIU Analyst - (20181561),04235 Citigroup Transaction Services (M),Banking/Financial,Banking / Finance,NaN,NaN
7,Telesales Customer Service - Call Centre,LEACOV SINGAPORE PTE. LTD.,Telesales/Telemarketing,Education,NaN,NaN
8,RMA Coordinator,Venture Electronics Services (Malaysia) Sdn Bhd,Sales - Corporate,Manufacturing / Production,NaN,NaN
9,Product Sales Engineer,Kumpulan Abex Sdn. Bhd.,Electronics Engineering > Product Management,Telecommunication,NaN,NaN


In [13]:
#Check unique obs of each column

print(df.Job_Title.nunique())
print(df.Company.nunique())
print(df.Job_Category.nunique())
print(df.Industry.nunique())


111
83
48
29


In [14]:
df.describe(include = "all")

,Job_Title,Company,Job_Category,Industry,Min_Salary,Max_Salary
count,123,123,123,121,70,70
unique,111,83,48,29,22,28
top,Personal Banker(Banca)-Top Bank - High Basic H...,Agensi Pekerjaan BGC Group (Malaysia) Sdn Bhd,Sales - Financial Services,Banking / Finance,2500,5000
freq,4,10,19,26,13,8


In [15]:
#view top ten job category

val = df['Job_Category'].value_counts()[:10]

Count_Category = pd.DataFrame(val).reset_index()
Count_Category.columns = ['Job Category', 'Counts'] 
Count_Category.style.background_gradient(cmap='Greens')

,Job Category,Counts
0,Sales - Financial Services,19
1,Retail Sales,11
2,Banking/Financial > Financial Planning/Wealth Management,9
3,Sales - Corporate,8
4,Marketing/Business Dev,8
5,Customer Service,7
6,IT - Software > Software Engineer/Programmer,5
7,Clerical/Administrative,5
8,Banking/Financial,4
9,Sales - Eng/Tech/IT,3


In [18]:
#plot pie chart for Job Category

a = Count_Category['Job Category']
b = Count_Category['Counts']

fig = px.pie(Count_Category, values= b, names=a, color_discrete_sequence=px.colors.sequential.algae[::-1], hole= 0.3,title='Top 10 Job Category')
fig.show()


In [19]:
#view top ten industry

ind = df['Industry'].value_counts()[:11]

Count_Industry = pd.DataFrame(ind).reset_index()
Count_Industry.columns = ['Industry', 'Counts'] 
Count_Industry.style.background_gradient(cmap='Blues')

,Industry,Counts
0,Banking / Finance,26
1,HR Mgmt / Consulting,13
2,Manufacturing / Production,11
3,Retail / Merchandise,8
4,Call Center / BPO,7
5,Education,6
6,Electrical & Electronics,6
7,Insurance,5
8,Transportation / Logistics,4
9,IT / Software,4


In [20]:
#Plot top ten industry

c = Count_Industry['Industry']
d = Count_Industry['Counts']

fig = px.pie(Count_Industry, values= d, names=c, color_discrete_sequence=px.colors.sequential.ice, hole= 0.3,title='Top 10 Industry')
fig.show()

In [21]:
# top ten minimum salary offered
min_s = df['Min_Salary'].value_counts()[:11]
Count_Min = pd.DataFrame(min_s).reset_index()
Count_Min.columns = ['Minimum Salary', 'Counts'] 
Count_Min.style.background_gradient(cmap='RdPu')

,Minimum Salary,Counts
0,2500,13
1,2000,9
2,1800,7
3,3000,7
4,1500,4
5,4000,4
6,2800,3
7,2700,3
8,2400,3
9,4800,2


In [22]:
# top ten maximum Salary offered 

max_s = df['Max_Salary'].value_counts()[:11]
Count_Max = pd.DataFrame(max_s).reset_index()
Count_Max.columns = ['Maximum Salary', 'Counts'] 
Count_Max.style.background_gradient(cmap='GnBu')

,Maximum Salary,Counts
0,5000,8
1,4000,7
2,3000,6
3,3500,5
4,8000,4
5,2800,4
6,2500,4
7,3100,3
8,5600,3
9,2400,2
